In [1]:
import numpy as np
import pandas as pd

In [2]:
# collect frequency data, starting with the whole dataset
pop='all_cnv'
ac,af=[pd.read_table('/oak/stanford/groups/mrivas/ukbb24983/cnv/panel/{0}_{1}.{2}'.format(c,pop,end))
        for (c,end) in [('ac','acount'),('af','afreq')]]
df=ac.merge(af[['ID','ALT_FREQS']], on='ID')
df=df.rename(columns={'ALT_FREQS':'AF_ALL', 'ALT_CTS':'AC_ALL', 'OBS_CT':'N_ALL'})
df['N_ALL'] /= 2
df['N_ALL'] = df['N_ALL'].astype(int)
df=df.drop(['REF','ALT'], axis=1)

# add data from each population
for pop in ['european','white_british','e_asian','s_asian','african']:
    short=pop.upper().replace('_','')[:3].replace('HI','BR') # hack for white british=WBR
    ac,af=[pd.read_table('/oak/stanford/groups/mrivas/ukbb24983/cnv/panel/{0}_{1}.{2}'.format(c,pop,end))
            for (c,end) in [('ac','acount'),('af','afreq')]]    
    df=df.merge(ac[['ID','ALT_CTS','OBS_CT']].merge(af[['ID','ALT_FREQS']], on='ID'), on='ID')
    df=df.rename(columns={'ALT_FREQS':'AF_'+short, 'ALT_CTS':'AC_'+short, 'OBS_CT':'N_'+short})
    df['N_'+short] /= 2
    df['N_'+short] = df['N_'+short].astype(int)
    
# did it work?
df.head(10)

,#CHROM,ID,AC_ALL,N_ALL,AF_ALL,AC_EUR,N_EUR,AF_EUR,AC_WBR,N_WBR,AF_WBR,AC_EAS,N_EAS,AF_EAS,AC_SAS,N_SAS,AF_SAS,AC_AFR,N_AFR,AF_AFR
0,1,1:723307-779322_-,2,482734,0.000002,1,358466,0.000001,1,333280,0.000002,0,2035,0.000000,0,7263,0.000000,1,6435,0.000078
1,1,1:723307-779322_+,1,482734,0.000001,1,358466,0.000001,0,333280,0.000000,0,2035,0.000000,0,7263,0.000000,0,6435,0.000000
2,1,1:752721-779322_-,1,482734,0.000001,0,358466,0.000000,0,333280,0.000000,0,2035,0.000000,0,7263,0.000000,0,6435,0.000000
3,1,1:723307-813034_+,1,482734,0.000001,1,358466,0.000001,1,333280,0.000002,0,2035,0.000000,0,7263,0.000000,0,6435,0.000000
4,1,1:723307-835499_+,1,482734,0.000001,1,358466,0.000001,1,333280,0.000002,0,2035,0.000000,0,7263,0.000000,0,6435,0.000000
5,1,1:723307-836684_+,4,482734,0.000004,2,358466,0.000003,2,333280,0.000003,1,2035,0.000246,0,7263,0.000000,0,6435,0.000000
6,1,1:723307-838555_+,2,482734,0.000002,0,358466,0.000000,0,333280,0.000000,0,2035,0.000000,0,7263,0.000000,0,6435,0.000000
7,1,1:752721-809876_-,1,482734,0.000001,0,358466,0.000000,0,333280,0.000000,0,2035,0.000000,1,7263,0.000069,0,6435,0.000000
8,1,1:723307-840753_-,1,482734,0.000001,0,358466,0.000000,0,333280,0.000000,0,2035,0.000000,0,7263,0.000000,0,6435,0.000000
9,1,1:723307-840753_+,1,482734,0.000001,0,358466,0.000000,0,333280,0.000000,1,2035,0.000246,0,7263,0.000000,0,6435,0.000000


In [3]:
# sort on position and write to file
df['SV']=df.apply(lambda x:(int(x['ID'].split(':')[0]), 
                            int(x['ID'].split(':')[1].split('-')[0]),
                            int(x['ID'].split(':')[1].split('-')[1].split('_')[0])), axis=1)
df=df.sort_values('SV').drop('SV',axis=1)
df.to_csv('ukb_cnv_reference.tsv.gz',sep='\t',index=None,header=True,compression='gzip')

In [4]:
df.sort_values('AC_ALL')[::-1][['ID','AF_ALL','AF_EUR','AF_AFR','AF_SAS','AF_EAS']].head(25)

,ID,AF_ALL,AF_EUR,AF_AFR,AF_SAS,AF_EAS
139116,8:137693624-137859277_-,0.015115,0.015391,0.001243,0.021135,0.000246
7441,1:25599041-25658903_-,0.014239,0.014845,0.002953,0.006402,0.000737
209579,15:22305089-22588209_+,0.013206,0.013222,0.016162,0.016729,0.010319
7350,1:25599041-25648747_-,0.012323,0.012897,0.000622,0.005852,0.000246
199842,14:19553431-20417653_+,0.008531,0.008695,0.002564,0.008123,0.004668
7458,1:25600747-25658903_-,0.008386,0.008774,0.000622,0.004337,0.000246
200081,14:20197824-20417653_+,0.008302,0.008269,0.002953,0.011703,0.009828
121758,7:142832341-142881275_-,0.007913,0.008390,0.000388,0.001377,0.000000
200063,14:20197824-20404501_+,0.007608,0.007698,0.004429,0.008605,0.008354
199901,14:19787984-20417653_+,0.006818,0.006653,0.004040,0.014663,0.004668
